In [ ]:
import pandas as pd

In [ ]:
#---------------------------------------------------
#
# Config *** TO UPDATE ***
#
#---------------------------------------------------

# *** TO UPDATE: change the team number with your own team number
AOIS_DEFINITION_FILE = "raw-data/team2/aoisDefinition.csv"

# This file should be automatically generated from "2. Event Detection"
GAZE_DATA_WITH_AOIS_AND_EVENTS_FILE = "data/gazeDataWithAOIsAndEvents.csv"

In [ ]:
#----------------------------------------------------------------------------------------
#
#
# 1. Mapping Fixations and Saccades to AOIs
#
#
#----------------------------------------------------------------------------------------

In [ ]:
# Read aois definition
aois_df = pd.read_csv(AOIS_DEFINITION_FILE)

In [ ]:
# Display aois definition
display(aois_df)

In [ ]:
# get list of AOIs
aois = aois_df["AOI"].tolist()
print(f'AOIs: {aois}')

In [ ]:
# Read gaze data with AOIs and events using pandas library
data = pd.read_csv(GAZE_DATA_WITH_AOIS_AND_EVENTS_FILE)
# set display.max_columns to none, to show all the columns when using head()
pd.set_option('display.max_columns', None)

In [ ]:
# Preview data
data.head()

In [ ]:
# Drop rows with GazeX or GazeY equals to NaN. This because these gazes are not mapped to any AOI
data = data.dropna(subset=['GazeX','GazeY'])

# Drop rows with both FixID and  SacID equal to NaN
data = data.dropna(subset=['FixID','SacID'], how='all')

# Keep only relevant colomns
fixationAndSaccadesDataWithAOIs = data[['Respondent','Timestamp','FixID', 'Fixation X', 'Fixation Y',
       'Fixation Start', 'Fixation End', 'Fixation Duration',
       'Fixation Dispersion', 'SacID', 'Saccade Start',
       'Saccade End', 'Saccade Duration', 'Saccade Amplitude',
       'Saccade Peak Velocity', 'Saccade Peak Acceleration',
       'Saccade Peak Deceleration', 'Saccade Direction']+aois].copy(deep=True)

# Map fixations and saccades to AOIs
"""
Goal: 
- Map fixations and saccades to AOIs
Context: 
- A fixation or saccade comprises several gazes.
- Following the NoteBook "1. Gaze Projections and AOIs Mapping", each gaze point is mapped to an AOI.
Problem: 
- Gazes belonging to the same fixation or saccade might not necessarily be mapped to the same AOI.
Solution: 
- When mapping fixations or saccades to AOIs, choose the AOI which is mapped to the majority of the gazes composing the fixation or the saccade
"""                                
fixationAndSaccadesDataWithAOIs = fixationAndSaccadesDataWithAOIs.groupby(['Respondent','FixID', 'Fixation X', 'Fixation Y',
       'Fixation Start', 'Fixation End', 'Fixation Duration',
       'Fixation Dispersion', 'SacID', 'Saccade Start',
       'Saccade End', 'Saccade Duration', 'Saccade Amplitude',
       'Saccade Peak Velocity', 'Saccade Peak Acceleration',
       'Saccade Peak Deceleration', 'Saccade Direction'],dropna=False).agg( {**{aoi: (lambda x: pd.Series.mode(x)[0]) for aoi in aois},**{'Timestamp': lambda x: x.iloc[0]}} ) 



# Flatten fixationAndSaccadesDataWithAOIs (i.e., a hierarchically indexed pandas.DataFrame)
fixationAndSaccadesDataWithAOIs.columns= [x for x in list(fixationAndSaccadesDataWithAOIs.columns.get_level_values(0))]
fixationAndSaccadesDataWithAOIs = fixationAndSaccadesDataWithAOIs.reset_index()

# Sort fixationAndSaccadesDataWithAOIs by Respondent and Timestamp
fixationAndSaccadesDataWithAOIs = fixationAndSaccadesDataWithAOIs.sort_values(by=['Respondent','Timestamp'])

# Reset index
fixationAndSaccadesDataWithAOIs = fixationAndSaccadesDataWithAOIs.reset_index(drop=True)

In [ ]:
# Preview fixation data
fixationAndSaccadesDataWithAOIs.head()

In [ ]:
# Export fixation data with AOIs to csv
fixationAndSaccadesDataWithAOIs.to_csv("data/fixationAndSaccadesDataWithAOIs.csv",  index=False)